In [2]:
# Access all the variable on .env file
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [3]:
from huggingface_hub import login
login(os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
# Chunk the dataset 
from langchain_community.document_loaders import TextLoader 

# Create list of path to your text files
file_paths = [f"./Typhoon_output/page{i}.txt" for i in range(1, 6)]

# Load the text files
documents = []
for path in file_paths:
    loader = TextLoader(path, encoding="utf-8")
    documents.extend(loader.load()) # Merge all documents into a single list

In [5]:
# Separate for each chunk by character spliter
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n\n", # split by new line
    chunk_size = 500,
    chunk_overlap = 50,
)
all_page = text_splitter.split_documents(documents) # split the content

Created a chunk of size 1133, which is longer than the specified 500
Created a chunk of size 696, which is longer than the specified 500


In [8]:
len(all_page) # check the number of chunks

21

In [ ]:
# Show the first chunk
for i in range(len(all_page)):
    print(f"Chunk {i+1}:")
    print(all_page[i].page_content)
    print("\n" + "="*70 + "\n") # separator between chunks  

Chunk 1:
สำนักงานคณะกรรมการกฤษฎีกา

พระราชกฤษฎีกา
ให้ใช้บทบัญญัติแห่งประมวลกฎหมายแพ่งและพาณิชย์
บรรพ ๑ และ ๒ ที่ได้ตรวจชำระใหม่

มีพระบรมราชโองการ ในพระบาทสมเด็จพระรามาธิบดีศรีสินทรมหาวชิราวุธ
พระมงกุฎเกล้าเจ้าอยู่หัว ให้ประกาศจงทราบทั่วกันว่า

จำเป็นแต่ได้ออกประกาศประมวลกฎหมายแพ่งและพาณิชย์ บรรพ ๑ และ ๒
แต่วันที่ ๑๑ พฤศจิกายน พระพุทธศักราช ๒๕๖๖ เป็นต้นมา ได้มีความเห็นเเนะนำมากหลาย เพื่อยัง
ประมวลกฎหมายนี้ให้สมบูรณ์ยิ่งขึ้น


Chunk 2:
และเมื่อได้พิจารณาโดยถ่องแท้แล้ว เห็นเป็นการสมควรให้ตรวจชำระบทบัญญัติ
ในบรรพ ๑ และ ๒ แห่งประมวลกฎหมายแพ่งและพาณิชย์ที่กล่าวมาข้างต้นนั้นใหม่

จึงทรงพระกรุณาโปรดเกล้าฯ สั่งว่า บทบัญญัติเดิมในบรรพ ๑ และ ๒ แห่งประมวล
กฎหมายแพ่งและพาณิชย์ที่ได้ประกาศไว้แต่ ณ วันที่ ๑๑ พฤศจิกายน พระพุทธศักราช ๒๔๖๖ นั้น
ให้ยกเลิกเสียสิ้น และใช้บทบัญญัติที่ได้ตรวจชำระใหม่ต่อท้ายพระราชกฤษฎีกานี้แทนสืบไป

ประกาศมา ณ วันที่ ๑๑ พฤศจิกายน พระพุทธศักราช ๒๕๖๘ เป็นปีที่ ๑๖ ใน
รัชกาลปัจจุบันนี้


Chunk 3:
ราชกิจจานุเบกษา เล่ม ๔๒/-/หน้า ๑/๑๑ พฤศจิกายน ๒๕๖๘


Chunk 4:
สำนักงานคณะกรรมการกฤษ

In [4]:
# Test Embedding by ollama 
from langchain_community.embeddings import OllamaEmbeddings 

In [5]:
embeddings = (
    OllamaEmbeddings(model="bge-m3:latest")
)

C:\Users\aueiiw\AppData\Local\Temp\ipykernel_31004\989446612.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  OllamaEmbeddings(model="bge-m3:latest")


In [6]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='bge-m3:latest', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [12]:
all_page_embeddings = embeddings.embed_documents([doc.page_content for doc in all_page]) # embed the chunk
# Show the first embedding
for i in range(len(all_page_embeddings)):
    print(f"Embedding {i+1}:")
    print(all_page_embeddings[i])
    print(f"Length of embedding {i+1}: {len(all_page_embeddings[i])}")
    print("\n" + "="*70 + "\n") # separator between embeddings

Embedding 1:
[-0.6156905293464661, 0.8616691827774048, -0.07111501693725586, -0.3188779950141907, 0.4007340371608734, -1.3438090085983276, 0.47901296615600586, -1.0332926511764526, -0.6441274285316467, 0.08691149950027466, -0.3960602879524231, 0.3439771831035614, -0.40383675694465637, 0.4630497097969055, -0.08871302753686905, 0.5550649762153625, 0.9598398208618164, -0.20860788226127625, 0.2546452581882477, 0.4506336450576782, -0.4824535846710205, -0.6772574782371521, -0.014532582834362984, 0.5721354484558105, -0.7625043988227844, 0.15935039520263672, -0.3857596814632416, -0.22933170199394226, 0.5058014988899231, 0.39453187584877014, 0.07763121277093887, 0.6412646770477295, -0.3880853056907654, 0.3682604432106018, 0.8681694865226746, -0.33203092217445374, -0.4564736783504486, -0.502341091632843, -0.9715069532394409, 0.2939091920852661, -0.8410719037055969, -0.1766224056482315, 0.2281838357448578, -2.017270565032959, 0.5474081039428711, 0.3998051583766937, 0.11854957044124603, -0.5687952

In [13]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(all_page, embeddings)

In [14]:
vector_store

In [16]:
# Querying the vector store 
query = "อธิบายเกี่ยวกับพระราชกฤษฎีกา?"
results = vector_store.similarity_search(query, k=5) # search for similar chunks
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(result.page_content)
    print("\n" + "="*70 + "\n") # separator between results

Result 1:
สำนักงานคณะกรรมการกฤษฎีกา

พระราชกฤษฎีกา
ให้ใช้บทบัญญัติแห่งประมวลกฎหมายแพ่งและพาณิชย์
บรรพ ๑ และ ๒ ที่ได้ตรวจชำระใหม่

มีพระบรมราชโองการ ในพระบาทสมเด็จพระรามาธิบดีศรีสินทรมหาวชิราวุธ
พระมงกุฎเกล้าเจ้าอยู่หัว ให้ประกาศจงทราบทั่วกันว่า

จำเป็นแต่ได้ออกประกาศประมวลกฎหมายแพ่งและพาณิชย์ บรรพ ๑ และ ๒
แต่วันที่ ๑๑ พฤศจิกายน พระพุทธศักราช ๒๕๖๖ เป็นต้นมา ได้มีความเห็นเเนะนำมากหลาย เพื่อยัง
ประมวลกฎหมายนี้ให้สมบูรณ์ยิ่งขึ้น


Result 2:
สำนักงานคณะกรรมการกฤษฎีกา

พระราชบัญญัติ
ให้ใช้บทบัญญัติบรรพ ๑
แห่งประมวลกฎหมายแพ่งและพาณิชย์ที่ได้ตรวจชำระใหม่
พ.ศ. ๒๕๓๕

ภูมิพลอดุลยเดช ป.ร.
ให้ว่า ณ วันที่ ๓๑ มีนาคม พ.ศ. ๒๕๓๕
เป็นปีที่ ๔๗ ในรัชกาลปัจจุบัน


Result 3:
พระบาทสมเด็จพระปรมินทรมหาภูมิพลอดุลยเดช มีพระบรมราชโองการโปรดเกล้าฯ
ให้ประกาศว่า
โดยที่เป็นการสมควรปรับปรุงบทบัญญัติบรรพ ๑ แห่งประมวลกฎหมายแพ่งและ
พาณิชย์ และบทบัญญัตลักษณะ ๒๓ สมาคม ของบรรพ ๓ แห่งประมวลกฎหมายแพ่งและพาณิชย์
จึงทรงพระกรุณาโปรดเกล้าฯ ให้ตราพระราชบัญญัติขึ้นไว้โดยคำแนะนำและยินยอม
ของสภานิติบัญญัติแห่งชาติ ทำหน้าที่รัฐสภา ดั

In [17]:
docs_and_score=vector_store.similarity_search_with_score(query)
docs_and_score

[(Document(id='2379a200-e5c0-44d0-bf3f-7ede7ed7e672', metadata={'source': './Typhoon_output/page1.txt'}, page_content='สำนักงานคณะกรรมการกฤษฎีกา\n\nพระราชกฤษฎีกา\nให้ใช้บทบัญญัติแห่งประมวลกฎหมายแพ่งและพาณิชย์\nบรรพ ๑ และ ๒ ที่ได้ตรวจชำระใหม่\n\nมีพระบรมราชโองการ ในพระบาทสมเด็จพระรามาธิบดีศรีสินทรมหาวชิราวุธ\nพระมงกุฎเกล้าเจ้าอยู่หัว ให้ประกาศจงทราบทั่วกันว่า\n\nจำเป็นแต่ได้ออกประกาศประมวลกฎหมายแพ่งและพาณิชย์ บรรพ ๑ และ ๒\nแต่วันที่ ๑๑ พฤศจิกายน พระพุทธศักราช ๒๕๖๖ เป็นต้นมา ได้มีความเห็นเเนะนำมากหลาย เพื่อยัง\nประมวลกฎหมายนี้ให้สมบูรณ์ยิ่งขึ้น'),
  np.float32(572.1749)),
 (Document(id='9f146b84-1e9a-4c4f-babc-27a40ccffcca', metadata={'source': './Typhoon_output/page2.txt'}, page_content='สำนักงานคณะกรรมการกฤษฎีกา\n\nพระราชบัญญัติ\nให้ใช้บทบัญญัติบรรพ ๑\nแห่งประมวลกฎหมายแพ่งและพาณิชย์ที่ได้ตรวจชำระใหม่\nพ.ศ. ๒๕๓๕\n\nภูมิพลอดุลยเดช ป.ร.\nให้ว่า ณ วันที่ ๓๑ มีนาคม พ.ศ. ๒๕๓๕\nเป็นปีที่ ๔๗ ในรัชกาลปัจจุบัน'),
  np.float32(631.79724)),
 (Document(id='f813326f-bcd5-4699-a752-77c58a798191', met

In [19]:
## Save the vector store to disk
vector_store.save_local("vector_store") # save the vector store to disk

In [8]:
# Load the vector store from disk
from langchain_community.vectorstores import FAISS          

vector_store = FAISS.load_local("vector_store", embeddings, allow_dangerous_deserialization=True) # load the vector store from disk

In [72]:
# Create Augmented Prompt to make RAG 

def augmented_prompt(query: str):
    # Get top 5 results from knowledge base
    results = vector_store.similarity_search(query, k=5)
    # Get the content from the results
    source_knowledge = "\n".join([result.page_content for result in results])
    # Fetch into an Augmented Prompt
    augmented_prompt = f"""บริบทต่อไปนี้คือข้อมูลที่ถูกดึงมาจากเอกสารเพื่อใช้ในการตอบคำถามของผู้ใช้:
    
    ------------------------------------------------
    บริบท: 
    {source_knowledge}
    ------------------------------------------------
    กรุณาตอบคำถามต่อไปนี้โดยใช้เฉพาะข้อมูลจากบริบทด้านบนเท่านั้น:
    คำถาม: {query}
    
    ข้อกำหนดในการตอบคำถาม: 
    - ตอบเป็นภาษาไทยสุภาพเท่านั้น
    - ห้ามใช้คำหยาบ คำไม่สุภาพ หรือคำที่ไม่เหมาะสม
    - หากไม่พบคำตอบในข้อมูลด้านบน ให้ตอบว่า: "ขออภัย ไม่พบข้อมูลเพียงพอที่จะตอบคำถามนี้"
    - หากคำถามไม่ชัดเจน ให้ตอบว่า: "คำถามไม่ชัดเจน กรุณาอธิบายเพิ่มเติม"
    - ห้ามตอบคำถามที่ไม่เกี่ยวข้องกับข้อมูลด้านบน
    - ห้ามใช้ข้อมูลนอกเหนือจากที่ให้มาในบริบท
    - ห้ามใช้ข้อมูลที่ไม่เกี่ยวข้องกับคำถาม
    - ห้ามใช้ข้อมูลที่ไม่ถูกต้องหรือไม่เป็นความจริง"""
    return augmented_prompt.strip()

In [17]:
import os 
import requests 

In [18]:
groq_api = os.getenv('GROQ_API_KEY')

if groq_api:
    print(f"Groq API Key exists and begins {groq_api[:8]}")
else:
    print("Groq API Key not set")

Groq API Key exists and begins gsk_l9Rz


In [53]:
# Link to check model => https://console.groq.com/docs/models
# Use the model 
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq # connect Groq API to use this model
from langchain.chat_models import init_chat_model

# Connect between Groq and LLama3
chat = ChatGroq(temperature=0, model_name="deepseek-r1-distill-llama-70b", streaming= True, api_key=groq_api)

In [54]:
# Control the prompt template
# Set role of the prompt template 

from langchain.schema import ( 
    AIMessage, # Chat history for AI response
    HumanMessage,# Input from the user 
    SystemMessage, # Set the role of model 
)    

In [62]:
# RAG with Augmented Prompt 

from langchain.schema import HumanMessage 

# Create a new user prompt 
prompt = HumanMessage(
    content=augmented_prompt("วันที่ ๓๑ มีนาคม พ.ศ. ๒๕๓๕") # Use the augmented prompt function
)
# Add the messages to the chat history
chat_history = [SystemMessage(content="You are a helpful assistant."), prompt]

response = chat(messages=chat_history) # Get the response from the model

print(response.content) # Print the response content

<think>
好的，我现在需要回答用户的问题：“วันที่ ๓๑ มีนาคม พ.ศ. ๒๕๓๕”。首先，我要仔细阅读提供的上下文，找到相关的信息。

在上下文中，第一段提到了“ให้ใช้บทบัญญัติบรรพ ๑ แห่งประมวลกฎหมายแพ่งและพาณิชย์ที่ได้ตรวจชำระใหม่ พ.ศ. ๒๕๓๕”和“ให้ว่า ณ วันที่ ๓๑ มีนาคม พ.ศ. ๒๕๓๕”。这表明在2535年3月31日，国王签署了相关法令。

接下来，我需要确认这个日期在上下文中是否有其他相关信息。查看后续内容，发现该日期主要用于指示法令的签署日期，没有其他额外信息。

因此，回答应该简明扼要，指出该日期是国王签署法令的日期，并且是该法令的重要部分。
</think>

วันที่ ๓๑ มีนาคม พ.ศ. ๒๕๓๕ เป็นวันที่พระบาทสมเด็จพระปรมินทรมหาภูมิพลอดุลยเดช ทรงลงพระปรมาภัยในพระราชบัญญัติให้ใช้บทบัญญัติบรรพ ๑ แห่งประมวลกฎหมายแพ่งและพาณิชย์ที่ได้ตรวจชำระใหม่ พ.ศ. ๒๕๓๕


In [73]:
# remove <think> and </think> tags
import re 

def remove_think_tags(text):
    return re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()

In [74]:
# function for RAG in Gradio
def generate_response(user_input):
    try: 
        prompt = HumanMessage(content=augmented_prompt(user_input))
        chat_history = [
            SystemMessage(content="You are a helpful assistant."),
            prompt
        ] 
        
        response = chat(messages=chat_history)
        
        cleaned_response = remove_think_tags(response.content)  # Clean the response]
        return cleaned_response 
    
    except Exception as e:
        return f"Error: {str(e)}"

In [80]:
# Create a Gradio interface
import gradio as gr
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(label="คำถาม", placeholder="เช่น วันที่ ๓๑ มีนาคม พ.ศ. ๒๕๓๕ หมายถึงอะไร?"),
    outputs=gr.Textbox(label="คำตอบจากระบบ"),
    title="Jass Chatbot",
    description="ระบบนี้จะดึงข้อมูลจากฐานความรู้เพื่อตอบคำถามของคุณอย่างแม่นยำ โดยใช้เฉพาะข้อมูลที่เกี่ยวข้องเท่านั้น"
)



In [81]:
iface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [82]:
iface.close()

Closing server running on port: 7861
